In [1]:
%config IPCompleter.use_jedi = False

In [2]:
import os
import json
import numpy as np
import requests
import time
import cv2
import base64
import random
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython.display import display, HTML
from k12libs.utils.nb_easy import RACEURL

In [3]:
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

In [4]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_INFERENCE

'http://116.85.5.40:9119/raceai/framework/inference'

In [5]:
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
API_POPMSG 

'http://116.85.5.40:9119/raceai/private/popmsg'

In [6]:
MODEL_TASK = 'zmq.asr.librispeech.inference'
MSGKEY = 'zmq.asr.test'
IMG_ROOT = '/raceai/data/audio_samples'

In [7]:
SAMPLE_FILES = [
    'example1.wav',
    'example2.flac',
]

## 调试

In [8]:
rawdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": %s,
                "dataset": {
                    "class_name": "raceai.data.RawRaceDataset",
                    "params": {
                        "data_prefix": "%s"
                    }
                }
            }
        }
    }
}'''
rawdata_conf = rawdata % (MODEL_TASK, MSGKEY, SAMPLE_FILES, IMG_ROOT)

### 输入

只需要修改: `data_source` 和 `data_prefix`

In [9]:
print(rawdata_conf)

{
    "task": "zmq.asr.librispeech.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.asr.test",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": ['example1.wav', 'example2.flac'],
                "dataset": {
                    "class_name": "raceai.data.RawRaceDataset",
                    "params": {
                        "data_prefix": "/raceai/data/audio_samples"
                    }
                }
            }
        }
    }
}


In [13]:
reqdata = eval(rawdata_conf)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
time.sleep(3)
resdata = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
resdata

[{'pigeon': {'msgkey': 'zmq.asr.test', 'user': '1', 'uuid': '100'},
  'task': 'zmq.asr.librispeech.inference',
  'errno': 0,
  'result': [{'id': 0, 'text': 'THE BIRCH CANOE SLID ON THE SMOOTH PLANKS'},
   {'id': 1, 'text': "MY FATHER HAS REVEALED THE CULPRIT'S NAME"}],
  'running_time': 2.038}]

### 输出

```json
[
    {
        "pigeon": {
            "msgkey": "zmq.asr.test",
            "user": "1",
            "uuid": "100"
        },
        "task": "zmq.asr.librispeech.inference",
        "errno": 0,
        "result": [
            {
                "id": 0,                                            // 传入图片的序号(data_source内容的顺序), 如果存在音频路径不存在的情况会有用
                "text": "THE BIRCH CANOE SLID ON THE SMOOTH PLANKS" // 音频解码后的文本
            },
            {
                "id": 1,
                "text": "MY FATHER HAS REVEALED THE CULPRIT'S NAME"
            }
        ],
        "running_time": 2.037
    }
]
````

## 单个Base64

In [33]:
# test_audio = '/hzcsk12/hzcsnote/assets/audio/example2.flac'
# test_audio = '/hzcsk12/hzcsnote/assets/audio/example.wav'
test_audio = '/hzcsk12/hzcsnote/assets/audio/test_audio_2.wav'

In [ ]:
def audio2b64(path):
    with open(path, 'rb') as fr:
        b4data = base64.b64encode(fr.read()).decode()
    if path.endswith('wav'):
        return 'data:audio/wav;base64,%s' % b4data
    elif path.endswith('flac'):
        return 'data:audio/flac;base64,%s' % b4data
    return b4data

reqdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user": "1",
                "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.Base64DataLoader",
            "params": {
                "data_source": "%s",
                "dataset": {
                    "class_name": "raceai.data.RawRaceDataset",
                    "params": {}
                }
            }
        }
    }
}''' % (MODEL_TASK, MSGKEY, audio2b64(test_audio))

reqdata = eval(reqdata)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
time.sleep(3)

In [43]:
resdata = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
print(json.dumps(resdata, indent=4))

[]
